In [ ]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.optimizers import Adam

# Hyperparameters
latent_dim = 256  # size of LSTM hidden state
num_samples = 10000  # number of sentence pairs

# These would be built from your dataset
# For simplicity, assume we already have:
# encoder_input_data: (num_samples, max_encoder_seq_length, num_encoder_tokens)
# decoder_input_data: (num_samples, max_decoder_seq_length, num_decoder_tokens)
# decoder_target_data: same as decoder_input_data but shifted left
# input_token_index, target_token_index: token→index mappings
# reverse_target_char_index: index→token mapping for inference

# -------- 1. ENCODER --------
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_lstm = LSTM(latent_dim, return_state=True)
_, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]  # the "meaning vector"

# -------- 2. DECODER --------
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# -------- 3. TRAINING MODEL --------
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer=Adam(), loss='categorical_crossentropy')

# Train (teacher forcing)
model.fit([encoder_input_data, decoder_input_data],
          decoder_target_data,
          batch_size=64,
          epochs=30,
          validation_split=0.2)

# -------- 4. INFERENCE MODELS --------
# Encoder model for inference
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder model for inference
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# -------- 5. DECODING LOOP --------
def decode_sequence(input_seq):
    # Encode input as state vectors
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence with start token
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['<start>']] = 1.0

    # Collect decoded tokens
    decoded_sentence = ''
    while True:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Pick token with highest probability
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' ' + sampled_char

        # Exit condition
        if sampled_char == '<end>' or len(decoded_sentence) > max_decoder_seq_length:
            break

        # Update target sequence
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]

    return decoded_sentence
